In [ ]:
import api_key

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = api_key.openai

In [20]:
model_name = 'gpt-3.5-turbo'

In [21]:
#%pip install langchain openai
#%pip install -U langchain_openai

In [22]:
from langchain.agents import create_agent

In [23]:
# basic connections to api
agent_test = create_agent(
    model=model_name,
    system_prompt="You are a helpful assistant who answers like Shakespeare would have."
)

In [24]:
agent_test.invoke({"messages":[{"role":"user", "content": "Tell me a joke about programmers."}]})

{'messages': [HumanMessage(content='Tell me a joke about programmers.', additional_kwargs={}, response_metadata={}, id='99497faa-ffae-42d9-9d95-f5b4e39d05ab'),
  AIMessage(content='Why, I prithee, let me regale thee with a jest about those learned in the ways of programming:\n\nWhy do programmers always mix up Christmas and Halloween?\n\nBecause Oct 31 == Dec 25!', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 30, 'total_tokens': 73, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-ChVaqlE8Biu3iNLsz9t7w0nNverEu', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--808bab51-758a-4bdc-802f-6a28c2d55235-0', usage_metadata={'input_t

## Model Tools

In [25]:
def get_weather(city: str) -> str:
    """Get weaather for the given city."""
    return f"It's always sunny in {city}!"

agent = create_agent(
    model=model_name,
    tools=[get_weather],
    system_prompt="You're a helpful weather agent."
)

In [26]:
agent.invoke(
    {"messages": [{
        "role": "user",
        "content": "what is the weather in toronto?"
    }]}
)

{'messages': [HumanMessage(content='what is the weather in toronto?', additional_kwargs={}, response_metadata={}, id='c42bf567-05e9-4be7-8711-8c396a42f9e6'),
  AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 14, 'prompt_tokens': 62, 'total_tokens': 76, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'id': 'chatcmpl-ChVarV1hVHESkvVtnVAK9qV656T8U', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--3ee3b516-bbba-4861-9428-33abd43080ed-0', tool_calls=[{'name': 'get_weather', 'args': {'city': 'Toronto'}, 'id': 'call_qk3E1xsGgp4xEj2tGkNiUvPd', 'type': 'tool_call'}], usage_metadata={'input_tokens': 62, 'output_tokens': 14, 'total_tokens': 76, 

In [27]:
SYSTEM_PROMPT ="""You are expert weather forecaster, who speaks like Shakespeare. 

You have access to two tools:
1. get_weather_for_locaition: use this to get the weather for specific location
2. get_user_location: use this get user's location

If a user asks for weather, make sure you know the location. If you can tell from the questions that they mean whereever they are,
user the get_user_location_tool to find their location.

"""

In [28]:
from dataclasses import dataclass
from langchain.tools import tool, ToolRuntime

@tool
def get_weather_for_location(city: str) -> str:
    """Get weather for the given city."""
    return f"It's always sunny in {city}!"

@dataclass
class Context:
    """Custom runtime context schema."""
    user_id: str

@tool
def get_user_location(runtime: ToolRuntime[Context]) -> str:
    """Retrive user informatio bsed on user ID."""
    user_id = runtime.context.user_id
    return "Toronto" if user_id =="1" else "Vancouver"

In [29]:
#  create you agent 
from langchain.chat_models import init_chat_model

model = init_chat_model(
    model=model_name,
    temperature=0.5,
    timeout=10,
    max_tokens=1000,    
)

In [30]:
# repsponse shcema defination
@dataclass
class ResponseFormat:
    """Response schema for the agent"""
    # alwasy required field
    shakespearean_weather_report: str
    # optional field
    weather_conditions: str | None = None

In [31]:
# adding memory to the agent
from langgraph.checkpoint.memory import InMemorySaver

checkpointer = InMemorySaver()

## can you database for persistance

In [32]:
from langchain.agents.structured_output import ToolStrategy

agent = create_agent(
    model=model,
    system_prompt=SYSTEM_PROMPT,
    tools=[get_user_location, get_weather_for_location],
    context_schema=Context,
    response_format=ToolStrategy(ResponseFormat),
    checkpointer=checkpointer
)

In [33]:
# running the agent
config = {"configurable": {"thread_id": "1"}} # if for this conversaton

response = agent.invoke(
    {"messages":[{
        "role":"user",
        "content": "what is the weather outside?"
    }]},
    config=config,
    context=Context(user_id='1')
)


In [34]:

print(response['structured_response'])

ResponseFormat(shakespearean_weather_report="It's always sunny in Toronto!", weather_conditions=None)


In [35]:
response = agent.invoke(
    {"messages":[{"role":"user", "content": "thank you!"}]},
    config=config,
    context=Context(user_id='1')
)
print(response['structured_response'])

ResponseFormat(shakespearean_weather_report="You're most welcome, fair citizen! Fare thee well.", weather_conditions=None)
